# Abstractions to Containerize Code/Model

## Setup Project

Create project to separate resources.

In [1]:
import mlrun
import requests

In [2]:
project = mlrun.get_or_create_project(name="berkeley-mlops", context=".")

> 2022-09-06 16:57:10,161 [info] created and saved project berkeley-mlops


## Simple Job

Something to run once to completion

In [4]:
%%writefile job.py
import mlrun
import pandas as pd
from sklearn.datasets import load_iris

def prep_data(context, label_column='label'):
    iris = load_iris()
    iris_dataset = pd.DataFrame(data=iris.data, columns=iris.feature_names)
    iris_labels = pd.DataFrame(data=iris.target, columns=[label_column])
    iris_dataset = pd.concat([iris_dataset, iris_labels], axis=1)
    
    context.logger.info(f'saving iris dataframe to {context.artifact_path}')
    context.log_dataset('iris_dataset', df=iris_dataset, format="csv", index=False)
    context.log_result('num_rows', iris_dataset.shape[0])

Overwriting job.py


In [5]:
job = mlrun.code_to_function(
    name="simple-job",
    filename="job.py",
    kind="job",
    image="mlrun/mlrun",
    handler="prep_data"
)

In [6]:
job.run()

> 2022-09-06 16:58:35,049 [info] starting run simple-job-prep_data uid=00182713765940a6af2d903ed0ead3bc DB=http://mlrun-api:8080
> 2022-09-06 16:58:35,222 [info] Job is running in the background, pod: simple-job-prep-data-rhlpx
> 2022-09-06 16:58:38,984 [info] saving iris dataframe to v3io:///projects/berkeley-mlops/artifacts
> 2022-09-06 16:58:39,077 [info] run executed, status=completed
final state: completed


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
berkeley-mlops,...d0ead3bc,0,Sep 06 16:58:38,completed,simple-job-prep_data,v3io_user=nickkind=jobowner=nickmlrun/client_version=1.0.4host=simple-job-prep-data-rhlpx,,,num_rows=150,iris_dataset


> 2022-09-06 16:58:41,548 [info] run executed, status=completed


## Simple Real-Time Function

An endpoint that is waiting for requests. Can scale to 0 if desired.

In [8]:
%%writefile rt.py
import random

def random_result(context, event):
    return {"result" : str(random.random())}

Overwriting rt.py


In [9]:
rt = mlrun.code_to_function(
    name="simple-rt",
    filename="rt.py",
    kind="nuclio",
    image="mlrun/mlrun",
    handler="random_result"
)

In [10]:
url = rt.deploy()

> 2022-09-06 17:00:24,249 [info] Starting remote function deploy
2022-09-06 17:00:24  (info) Deploying function
2022-09-06 17:00:24  (info) Building
2022-09-06 17:00:24  (info) Staging files and preparing base images
2022-09-06 17:00:24  (info) Building processor image
2022-09-06 17:01:29  (info) Build complete
2022-09-06 17:02:09  (info) Function deploy complete
> 2022-09-06 17:02:09,753 [info] successfully deployed function: {'internal_invocation_urls': ['nuclio-berkeley-mlops-simple-rt.default-tenant.svc.cluster.local:8080'], 'external_invocation_urls': ['berkeley-mlops-simple-rt-berkeley-mlops.default-tenant.app.us-sales-350.iguazio-cd1.com/']}


In [11]:
url

'http://berkeley-mlops-simple-rt-berkeley-mlops.default-tenant.app.us-sales-350.iguazio-cd1.com/'

In [17]:
requests.get(url).json()

{'result': '0.30656081561393633'}

## Simple Model Deployment

An endpoint specifically designed for model serving that is waiting for requests. Allows for specifying a model's behavior using a Python class. Can scale to 0 if desired.

In [18]:
%%writefile model.py
import mlrun

from cloudpickle import load
from sklearn.datasets import load_iris
import numpy as np

import warnings

warnings.filterwarnings("ignore")

class ClassifierModel(mlrun.serving.V2ModelServer):
    def load(self):
        """load and initialize the model and/or other elements"""
        model_file, extra_data = self.get_model(".pkl")
        self.model = load(open(model_file, "rb"))

    def predict(self, body: dict) -> list:
        """Generate model predictions from sample."""
        feats = np.asarray(body["inputs"])
        result: np.ndarray = self.model.predict(feats)
        return result.tolist()

Overwriting model.py


In [22]:
serve = mlrun.code_to_function(
    name="model-server",
    filename="model.py",
    kind="serving",
    image="mlrun/mlrun"
)

serve.add_model("my_model", model_path="https://s3.wasabisys.com/iguazio/models/iris/model.pkl", class_name="ClassifierModel")

In [23]:
url = serve.deploy()

> 2022-09-06 17:05:07,875 [info] Starting remote function deploy
2022-09-06 17:05:08  (info) Deploying function
2022-09-06 17:05:08  (info) Building
2022-09-06 17:05:08  (info) Staging files and preparing base images
2022-09-06 17:05:08  (info) Building processor image
2022-09-06 17:06:13  (info) Build complete
2022-09-06 17:07:09  (info) Function deploy complete
> 2022-09-06 17:07:10,088 [info] successfully deployed function: {'internal_invocation_urls': ['nuclio-berkeley-mlops-model-server.default-tenant.svc.cluster.local:8080'], 'external_invocation_urls': ['berkeley-mlops-model-server-berkeley-mlops.default-tenant.app.us-sales-350.iguazio-cd1.com/']}


In [24]:
url

'http://berkeley-mlops-model-server-berkeley-mlops.default-tenant.app.us-sales-350.iguazio-cd1.com/'

In [26]:
requests.get(url).json()

{'name': 'ModelRouter', 'version': 'v2', 'extensions': []}

In [27]:
requests.get(f"{url}/v2/models").json()

{'models': ['my_model']}

In [39]:
data = {
    "inputs" : [
        [5.1, 3.5, 1.4, 0.2],
        [4.9, 3.0, 1.4, 0.2],
        [2.6, 3.4, 3.2, 4.3]
    ]
}

In [40]:
requests.post(f"{url}/v2/models/my_model/predict", json=data).json()

{'id': 'e60e6125-d80d-44af-b502-98925e0053ad',
 'model_name': 'my_model',
 'outputs': [0, 0, 1]}